In [1]:
from torch import nn
#定义VGG16模型,
class VGG16Model(nn.Module):#该模型继承自PyTorch的nn.Module基类
    def __init__(self, num_classes, conv_arch, input_channels=3):#当你创建一个类的新实例时,初始化对象的状态的函数
        super(VGG16Model, self).__init__()#调用父类的init函数；再子类init中要调用父类的init函数
        #self是一个对实例对象本身的引用
        self.num_classes = num_classes # 模型的输出类别数量
        self.conv_arch = conv_arch # 卷积层的架构，通常是一个包含多个元组的列表，每个元组包含卷积层的数量和输出通道数。
        self.input_channels = input_channels # 输入图像的通道数，默认为3（对于RGB图像）。
        self.features = self.make_layers()#调用make_layers函数创建卷积层部分。
        self.avgpool = nn.AdaptiveAvgPool2d((3, 3))#使用自适应平均池化层将特征图的大小调整为3x3。
        self.classifier = nn.Sequential(  
            nn.Flatten(),  
            nn.Linear(64,6422528),  # 假设这是正确的输入特征数量  
            nn.ReLU(inplace=True),  
            nn.Dropout(0.5),  
            nn.Linear(512, 512),  
            nn.ReLU(inplace=True),  
            nn.Dropout(0.5),  
            nn.Linear(512, self.num_classes)  
        )  
    def make_layers(self):
            layers = []
            for (num_convs, out_channels) in self.conv_arch:#对于conv_arch中的每个(num_convs, out_channels)元组，它将添加num_convs个卷积层，每个卷积层都有self.input_channels个输入通道和out_channels个输出通道。然后添加一个ReLU激活函数，并更新self.input_channels为out_channels。最后，添加一个最大池化层以减小特征图的大小。
                for _ in range(num_convs):
                    layers.append(nn.Conv2d(self.input_channels, out_channels, kernel_size=3, padding=1))
                    layers.append(nn.ReLU(inplace=True))
                    self.input_channels = out_channels
                    layers.append(nn.MaxPool2d(kernel_size=1, stride=1))
            return nn.Sequential(*layers)
    def forward(self, input):#前向传播函数
            vggfeatures = self.features(input)#将输入图像传递给卷积层部分以提取特征。
            output = self.classifier(vggfeatures)#将提取的特征传递给分类器部分以获取输出。
            return output#即模型的预测结果。

In [2]:
import os  
import torch  
from torch.utils.data import Dataset, DataLoader  
from torchvision import transforms as T  
from PIL import Image 
import torch.nn.functional as F
  
class CustomImageDataset(Dataset):  
    def __init__(self, root_dir, transform=None):  
        self.root_dir = root_dir  
        self.transform = transform  
        self.image_files = []  
        self.labels = []  
  
        # 遍历文件夹并获取图片文件和对应的标签  
        for class_name in os.listdir(root_dir):  
            class_dir = os.path.join(root_dir, class_name)  
            if os.path.isdir(class_dir):  
                for file_name in os.listdir(class_dir):  
                    if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):  
                        self.image_files.append(os.path.join(class_dir, file_name))  
                        self.labels.append(class_name)  # 或者使用某种方式将类名映射为整数标签  
  
    def __len__(self):  
        return len(self.image_files)  
  
    def __getitem__(self, idx):  
        image_path = self.image_files[idx]  
        image = Image.open(image_path).convert('RGB')  # 函数返回一个图像对象，该对象包含了图像的数据和元数据。
  
        if self.transform:  
            image = self.transform(image)  
  
        label = self.labels[idx]  # 如果需要整数标签，请在这里进行转换  
        return image, label  
    
def save_model(model, epoch, save_path='/kaggle/working/model.pth'):  
    torch.save({'epoch': epoch,  
                'model_state_dict': model.state_dict(),  
                'optimizer_state_dict': optimizer.state_dict(),  
                'loss': loss}, save_path)  
    print(f'Model saved at epoch {epoch}')  
def tuple_to_onetensor(string_labels,label_to_idx):
    # 将字符串标签转换为整数  
    int_labels = [label_to_idx[label] for label in string_labels]  
  
    # 将整数标签转换为tensor（如果它们还不是）  
    int_labels_tensor = torch.tensor(int_labels, dtype=torch.long)  
  
    # 进行独热编码（假设num_classes是类别数量）  
    num_classes = len(label_to_idx)  
    one_hot_tensor = F.one_hot(int_labels_tensor, num_classes=num_classes)
    return one_hot_tensor
  

In [4]:
import torch  
import torch.nn as nn  
import torch.optim as optim  

# 超参数和配置  
num_classes = 10  # 假设我们有10个类别
batch_size = 64  
learning_rate = 0.001  
num_epochs = 5  

# 定义标签到整数的映射  
label_to_idx = {'105': 0, '118': 1, '130': 2, '169': 3, '185': 4, '197': 5, '209': 6, '222': 7, '234': 8, '97': 9} 

# 定义VGG16模型的conv_arch（这里只是一个简化的示例）  
conv_arch = [(2, 32), (2, 64), (3, 128)]  

# 数据预处理  
transform = T.Compose([  
    T.Resize(256),  
    T.CenterCrop(224),  
    T.ToTensor(),  
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  
])  

# 假设你的图片文件夹路径是'./local_images'  
trainset = CustomImageDataset(root_dir='/kaggle/input/classifybearing-ads-data/CWT/train', transform=transform)  
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valset = CustomImageDataset(root_dir='/kaggle/input/classifybearing-ads-data/CWT/val', transform=transform)  
valloader = DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=2)
testset = CustomImageDataset(root_dir='/kaggle/input/classifybearing-ads-data/CWT/test', transform=transform)  
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)
# 实例化模型  
model = VGG16Model(num_classes=num_classes, conv_arch=conv_arch)  

# 定义损失函数和优化器  
criterion = nn.CrossEntropyLoss()  
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)  

In [ ]:
import time  
import torch 
 

#训练模型  
start_time = time.time()  
elapsed_time = 0  
for epoch in range(num_epochs):  
    model.train()  # 设置模型为训练模式  
    running_loss = 0.0  # 用于记录当前epoch的训练损失  
  
    for batch_idx, (inputs, labels) in enumerate(trainloader):  
        optimizer.zero_grad()  # 清零梯度缓存  
        outputs = model(inputs)  # 前向传播 
        labels0=tuple_onetensor(labels,label_to_idx)
        loss = criterion(outputs, labels0)  # 计算损失  
        loss.backward()  # 反向传播  
        optimizer.step()  # 更新权重  
  
        running_loss += loss.item()  # 累加训练损失  
  
        # 显示进度  
        if batch_idx % 10 == 0:  # 每10个batch打印一次  
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(  
                epoch, batch_idx * len(inputs), len(trainloader.dataset),  
                100. * batch_idx / len(trainloader), loss.item()))  
  
    # 计算平均训练损失  
    train_loss = running_loss / len(trainloader.dataset)  
  
    # 验证模型  
    model.eval()  # 设置模型为评估模式  
    val_loss = 0.0  
    with torch.no_grad():  # 不需要计算梯度  
        for val_inputs, val_labels in valloader:  
            val_outputs = model(val_inputs)  
            labels0=tuple_onetensor(val_labels,label_to_idx)
            val_loss += criterion(val_outputs, val_labels0).item()  
  
    # 计算平均验证损失  
    val_loss /= len(valloader.dataset)  
  
    # 显示验证损失和所用时间  
    end_time = time.time()  
    elapsed_time = end_time - start_time  
    print('Epoch {} - Train loss: {:.4f}, Val loss: {:.4f}, Time: {:.2f}s'.format(  
        epoch, train_loss, val_loss, elapsed_time))  
    # 检查是否超过了5小时  
    if elapsed_time > 5 * 60 * 60:  # 5 hours in seconds  
        print('Training time exceeded 5 hours. Saving model and exiting.')  
        save_model(model, epoch)  
        break  
      
    # 可以在每个epoch结束时保存模型，或者根据验证损失来保存最佳模型  
# 如果循环正常结束（没有因为时间而中断），也可以在这里保存最终模型  
save_model(model, num_epochs - 1) 
print('Training complete.')